In [1]:
#@everywhere using DelimitedFiles
#@everywhere using Revise, MDToolbox
#using DelimitedFiles
using Revise, MDToolbox
ENV["COLUMNS"]=110; #display width for MDToolbox

┌ Info: Precompiling MDToolbox [7965e2f0-a35b-11e8-0207-19c030dd2def]
└ @ Base loading.jl:1278


In [2]:
receptor = mdload("9rat.pdb")

1x951 TrjArray{Float64,Int64}
| A                          | A                          |  …   A                          |
| 1LYS                       | 1LYS                       |  …   124VAL                     |
| 1N                         | 2CA                        |  …   951OXT                     |
|    27.84    45.62    53.14 |    27.54    44.35    52.53 |  …       4.33    36.08    31.87 |

In [3]:
receptor = MDToolbox.set_radius(receptor)

1x951 TrjArray{Float64,Int64}
| A                          | A                          |  …   A                          |
| 1LYS                       | 1LYS                       |  …   124VAL                     |
| 1N                         | 2CA                        |  …   951OXT                     |
|    27.84    45.62    53.14 |    27.54    44.35    52.53 |  …       4.33    36.08    31.87 |

In [4]:
receptor = MDToolbox.compute_sasa(receptor, 1.4)

1x951 TrjArray{Float64,Int64}
| A                          | A                          |  …   A                          |
| 1LYS                       | 1LYS                       |  …   124VAL                     |
| 1N                         | 2CA                        |  …   951OXT                     |
|    27.84    45.62    53.14 |    27.54    44.35    52.53 |  …       4.33    36.08    31.87 |

In [27]:
result = MDToolbox.dock_multimer(receptor, nfold=5)

"step 1" = "step 1"
"step 2" = "step 2"
"step 3" = "step 3"
nx = 146
ny = 152
nz = 142
"step 4" = "step 4"
"step 4.1" = "step 4.1"
"step 4.2" = "step 4.2"
"step 5" = "step 5"
0.000000 
0.174533 
0.349066 
0.523599 
0.698132 
0.872665 
1.047198 
1.221730 
1.396263 
1.570796 
1.745329 
1.919862 
2.094395 
2.268928 
2.443461 
2.617994 
2.792527 
2.967060 
3.141593 
3.316126 
3.490659 
3.665191 
3.839724 
4.014257 
4.188790 
4.363323 
4.537856 
4.712389 
4.886922 
5.061455 
5.235988 
5.410521 
5.585054 
5.759587 
5.934119 
6.108652 
6.283185 


(receptor = 1x951 TrjArray{Float64,Int64}
| A                          | A                          |  …   A                          |
| 1LYS                       | 1LYS                       |  …   124VAL                     |
| 1N                         | 2CA                        |  …   951OXT                     |
|   -10.01    14.95    10.97 |    -9.93    14.34     9.67 |  …      11.50    -6.31    -2.51 |, ligand = 1x951 TrjArray{Float64,Int64}
| A                          | A                          |  …   A                          |
| 1LYS                       | 1LYS                       |  …   124VAL                     |
| 1N                         | 2CA                        |  …   951OXT                     |
|   -12.51    21.50    10.97 |   -11.91    21.39     9.67 |  …      14.36    35.38    -2.51 |, score = 1895.0000000000123)

In [35]:
result.score

1895.0000000000123

In [36]:
result.ligand

1x951 TrjArray{Float64,Int64}
| A                          | A                          |  …   A                          |
| 1LYS                       | 1LYS                       |  …   124VAL                     |
| 1N                         | 2CA                        |  …   951OXT                     |
|   -12.51    21.50    10.97 |   -11.91    21.39     9.67 |  …      14.36    35.38    -2.51 |

In [37]:
writepdb("result_receptor.pdb", result.receptor)
writepdb("result_ligand.pdb", result.ligand)

In [38]:
com1 = centerofmass(result.receptor)
com2 = centerofmass(result.ligand)

L = [com2.xyz[1] - com1.xyz[1], com2.xyz[2] - com1.xyz[2]]
L_norm = sqrt(L[1]^2 + L[2]^2)

nfold = 2.0
alpha = (180.0 - (360.0/nfold))/2.0
alpha = (alpha / 360.0) * 2.0 * pi

d_norm = L_norm / (2.0*cos(alpha))

13.41640786499874

In [42]:
nfold = 5
beta = 360.0/nfold
beta = (beta / 360.0) * 2.0 * pi
R = [cos(beta) -sin(beta) 0.0; sin(beta) cos(beta) 0.0; 0.0 0.0 1.0]

complex = deepcopy(result.receptor)
complex.xyz[:, 1:3:end] .= complex.xyz[:, 1:3:end] .+ d_norm

for i = 2:nfold
    r = rotate_with_matrix(complex[end, :], R)
    complex = [complex; r]
end

In [43]:
writepdb("complex.pdb", complex)

In [44]:
pwd()

"/Users/yasu/gdrive/notebooks/docking_multimer"